In [11]:
import os
import csv
import mysql.connector
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from collections import defaultdict

In [12]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="root",
  database="digicart"
)

In [13]:
mycursor = mydb.cursor()

In [14]:
pr_file_path="C:/Program Files/Apache Software Foundation/Tomcat 7.0/webapps/DigiCart/"
os.chdir('C:/Program Files/MongoDB/Server/4.2/bin')
mycursor.execute("select username,id,viewCount as count from searchhistory")
views = mycursor.fetchall()
headers = [col[0] for col in mycursor.description]
viewWriter = open('C:/Users/Chait/Desktop/mongosearchdata_train.csv', 'w', newline="")
myFile = csv.writer(viewWriter)
myFile.writerow(headers)
myFile.writerows(views)
viewWriter.close()

with open("C:/Users/Chait/Desktop/mongosearchdata_train.csv", "r") as f:
    reader = csv.DictReader(f, delimiter=',')
    with open("C:/Users/Chait/Desktop/mongosearchdata_test.csv", "w",newline='') as f_out:
        writer = csv.DictWriter(f_out, fieldnames=reader.fieldnames, delimiter=",")
        for row in reader:
            writer.writerow(row)
            
file_path = os.path.expanduser('C:/Users/Chait/Desktop/mongosearchdata_test.csv')

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='user item rating', sep=',')

#data = Dataset.load_from_file(file_path, reader=reader)

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    #print(top_n)

    return top_n

# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
#print(testset)
predictions = algo.test(testset)
#print(predictions)

top_n = get_top_n(predictions, n=5)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    
out = open(pr_file_path+'/output.csv', 'w',newline='')
output=csv.writer(out)

for uid, user_ratings in top_n.items():
    output.writerow([uid, [iid for (iid, _) in user_ratings]])
    
out.close()


palash ['AVpg8GjjLJeJML43Uplr', 'AVpivePGilAPnD_xHH0l']
chaitanya ['AVpf3txeLJeJML43FN82', 'AVpfuJ4pilAPnD_xhDyM', 'AVpf0Nyo1cnluZ0-rzhu', 'AVpfgGwSLJeJML43-VCn', 'AVphRsH11cnluZ0-Agzr']
harika ['AVpf0Nyo1cnluZ0-rzhu', 'AVphRsH11cnluZ0-Agzr', 'AVpivePGilAPnD_xHH0l', 'AVphUeKeilAPnD_x3-Be', 'AVpf3txeLJeJML43FN82']
dd ['AVpf3txeLJeJML43FN82', 'AVpfuJ4pilAPnD_xhDyM', 'AVpfVI9wilAPnD_xZxH-', 'AVpfjP9uilAPnD_xdy6-', 'AVpg8GjjLJeJML43Uplr']
123 ['AVphRsH11cnluZ0-Agzr', 'AVpf3txeLJeJML43FN82', 'AVsRhL-fQMlgsOJE6zvB', 'AVphUeKeilAPnD_x3-Be', 'AVpf0Nyo1cnluZ0-rzhu']
palashjain ['AVpfVI9wilAPnD_xZxH-', 'AVpf3txeLJeJML43FN82', 'AVsRhL-fQMlgsOJE6zvB', 'AVpe9BGF1cnluZ0-annA', 'AV4GShTMvKc47QAVrFsX']
